In [1]:
## First Stage of Preprocessing

import json 
import numpy as np
import os

from PIL import Image
import tifffile as tiff

<h3> Define Functions </h3>

In [2]:
def stack_bands(path, product_id):

    # Define the list of bands to stack
    band_list = ['B02.tif', 'B03.tif', 'B08.tif', 'B11.tif']

    # Initialize an empty list to store the data
    stacked_data = []
    try:
        # Loop through each band and stack them
        for band_file in band_list:
            band_path = os.path.join(path, band_file)
            image = tiff.imread(band_path)
            
            #to make sure every image has 512 * 512(Resizing is done only to reduce the total size of the dataset)
            pil_image = Image.fromarray(image)
            resized_image = pil_image.resize((256, 256))

            resize_array = np.array(resized_image)
            stacked_data.append(resize_array)

        # Stack the data along a new axis
        stacked_data = np.stack(stacked_data, axis=-1)

        # Save the stacked data as a new .tif file
        output_path = os.path.join(path, 'stack.tif')
        tiff.imsave(output_path, stacked_data)
    except:
        print(f"file with no data: {path}")
        pass

In [3]:
def product_name(path):
    json_data=open(path+"/stac.json", "rb")
    jdata = json.load(json_data)
    
    return  jdata['id']

In [4]:
def check_file_in_folder(folder_path, file_name):
    file_path = os.path.join(folder_path, file_name)
    if os.path.isfile(file_path):
        return True
    else:
        return False

In [5]:
# This functions deletes a folder and its containds if no spectral band is included
import shutil

def remove_empty_folders(path):
    print(f"Deleted: {path}")
    #os.remove(path) # to remove any specific file like jpeg, png, tif
    shutil.rmtree(path) # to remove any specific folder

In [6]:
# Create a list with all the folders
flist = []
rootdir = 'C:/Users/amitr/OneDrive/Desktop/Fllod Detection/sen12floods_s2_source'
for file in os.listdir(rootdir):
    d = os.path.join(rootdir, file)
    if os.path.isdir(d):
        flist.append(d)
        
print(f"The number of folders are currently = {len(flist)}")

The number of folders are currently = 1949


In [ ]:
#####################################################################################################################

In [7]:
# Iterate through all folders and create a new image with 4 spectral bands, namely Band2, Band3, Band8 and Band 11. 
# These bands correspend to blue, green, infrared respectively and SWIR

for folder_path in flist:
    product_id = product_name(folder_path)
    stack_bands(folder_path, product_id)

In [ ]:
stack = 0
pixel = 0

for folder_path in flist:
    contain = check_file_in_folder(folder_path, 'stack.tif') #check if the folder contains stack image
    if (contain == False):
        remove_empty_folders(folder_path)
        stack += 1
    else:
        stack_path = os.path.join(folder_path, 'stack.tif')
        stack_image = tiff.imread(stack_path)
        if(np.argmax(stack_image) == 0):
            remove_empty_folders(folder_path)
            pixel += 1

print(f"Folder not cntaining stack file:  {stack}")
print(f"stack file contain only 0 pixel:  {pixel}")

In [ ]:
#to remove stack file
# stack = 0

# for folder_path in flist:
#     contain = check_file_in_folder(folder_path, 'stack.tif') #check if the folder contains stack image
#     if (contain):
#         stack_path = os.path.join(folder_path, 'stack.tif')
#         remove_empty_folders(stack_path)
#         stack += 1

# print(f"Folder not cntaining stack file:  {stack}")

In [ ]:
# Create a list with all the folders after removing all the useless folder
flist = []
rootdir = 'C:/Users/amitr/OneDrive/Desktop/Fllod Detection/sen12floods_s2_source'
for file in os.listdir(rootdir):
    d = os.path.join(rootdir, file)
    if os.path.isdir(d):
        flist.append(d)
        
print(f"The number of folders after the prepocessing = {len(flist)}")

In [ ]:
output_path = 'C:/Users/amitr/OneDrive/Desktop/Fllod Detection/sen12floods_s2_source/sen12floods_s2_source_0001_2018_12_18/stack.tif'

saved_stacked_data = tiff.imread(output_path)
stacked_shape = saved_stacked_data.shape

print("Shape of the saved stacked TIFF file:", stacked_shape)

In [ ]:
##The first stage of preprocessing has been completed.